In [1]:
!wget https://www.dropbox.com/sh/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca?dl=0

--2019-04-17 07:47:43--  https://www.dropbox.com/sh/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca [following]
--2019-04-17 07:47:43--  https://www.dropbox.com/sh/raw/2idnozs4741hzkr/AABXTlPlU-1f4L_VdS4wY06ca
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc18aa119cfb47443751c34bba9.dl.dropboxusercontent.com/zip_by_token_key?key=AfLl1D0ZvG88tWFu6zixXpmz1NgO9qn7rMtVfowswDjYRq0oiPtXtTIKEw6rmam6-GT7P0sg9otBCbI3OHswOsN3Kfo3R3gOcjsuY8AXYtx2knqdyszgxR7XjH5zK3yluQUKONEsyueAFYt24hwuGrvelJyS61N6hfCFf0wKAVaSt9iLXvlNZZHTzdDRXYIQ363csrkw3oFwqUgV_lbGdQoyabXAcN0dbqhVblPYrW0-L6-PN8Ef-wzTyIvsuVsnWHY [following]
--2019-04-17 07:47:43--  https://ucc

In [2]:
!unzip AABXTlPlU-1f4L_VdS4wY06ca?dl=0

Archive:  AABXTlPlU-1f4L_VdS4wY06ca?dl=0
mapname:  conversion of  failed
  inflating: images/horses/horse-146.jpg  
  inflating: images/humans/rider-74.jpg  
  inflating: images/dogs/dog.120.jpg  
  inflating: images/dogs/dog.82.jpg  
  inflating: images/cats/cat.75.jpg  
  inflating: images/horses/horse-45.jpg  
  inflating: images/humans/rider-68.jpg  
  inflating: images/humans/rider-185.jpg  
  inflating: images/horses/horse-27.jpg  
  inflating: images/horses/horse-36.jpg  
  inflating: images/cats/cat.45.jpg  
  inflating: images/humans/rider-192.jpg  
  inflating: images/humans/rider-81.jpg  
  inflating: images/dogs/dog.115.jpg  
  inflating: images/humans/rider-154.jpg  
  inflating: images/horses/horse-158.jpg  
  inflating: images/humans/rider-95.jpg  
  inflating: images/humans/rider-98.jpg  
  inflating: images/horses/horse-127.jpg  
  inflating: images/cats/cat.200.jpg  
  inflating: images/horses/horse-73.jpg  
  inflating: images/humans/rider-73.jpg  
  inflating: image

In [3]:
!ls

'AABXTlPlU-1f4L_VdS4wY06ca?dl=0'   images   Images   sample_data


In [4]:
import os
folders = os.listdir("images")
print(folders)

['horses', 'cats', 'dogs', 'humans']


In [0]:
image_data = []
labels = []

label_dict = {
    'cats':0,
    'dogs':1,
    'horses':2,
    'humans':3
}

In [6]:
from keras.preprocessing import image

for ix in folders:
  path = os.path.join('images',ix)
  for im in os.listdir(path):
    img = image.load_img(os.path.join(path,im), target_size=((224,224)))
    img_array = image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])

Using TensorFlow backend.


In [7]:
print(len(image_data), len(labels))

808 808


In [0]:
import random
combined = list(zip(image_data, labels))
random.shuffle(combined)

image_data[:], labels[:] = zip(*combined)

In [9]:
print(labels[:5])

[0, 1, 1, 3, 1]


In [10]:
import numpy as np

X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape, Y_train.shape)

(808, 224, 224, 3) (808,)


In [11]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape, Y_train.shape)

(808, 224, 224, 3) (808, 4)


In [0]:
#Create the Resnet50 model
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model

import matplotlib.pyplot as plt

In [13]:
model = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________

In [14]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256, activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4, activation='softmax')(d1)

model_new = Model(input=model.input, output=fc2)
model_new.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
adam = Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [16]:
for ix in range(len(model_new.layers)):
  print(ix, model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x7f92225295f8>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x7f91d98c42e8>
2 <keras.layers.convolutional.Conv2D object at 0x7f91d98c4080>
3 <keras.layers.normalization.BatchNormalization object at 0x7f91d98c46a0>
4 <keras.layers.core.Activation object at 0x7f91d98c47f0>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x7f91d9821278>
6 <keras.layers.pooling.MaxPooling2D object at 0x7f91d9821b00>
7 <keras.layers.convolutional.Conv2D object at 0x7f91d9867908>
8 <keras.layers.normalization.BatchNormalization object at 0x7f91bc6fa3c8>
9 <keras.layers.core.Activation object at 0x7f91bc6fa470>
10 <keras.layers.convolutional.Conv2D object at 0x7f91bc718358>
11 <keras.layers.normalization.BatchNormalization object at 0x7f91bc730320>
12 <keras.layers.core.Activation object at 0x7f91bc6ef630>
13 <keras.layers.convolutional.Conv2D object at 0x7f91bc626470>
14 <keras.layers.convolutional.Conv2D object at 0x7f91bc5951d0>
15 <keras.lay

In [17]:
for ix in range(169):
  model_new.layers[ix].trainable = False
  
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [20]:
hist = model_new.fit(X_train, Y_train, shuffle=True, batch_size=16, epochs=5, validation_split=0.20)

Train on 646 samples, validate on 162 samples
Epoch 1/5
646/646 [==============================] - 4s 6ms/step - loss: 0.0945 - acc: 0.9783 - val_loss: 0.1402 - val_acc: 0.9568
Epoch 2/5
646/646 [==============================] - 4s 6ms/step - loss: 0.1116 - acc: 0.9721 - val_loss: 0.1319 - val_acc: 0.9568
Epoch 3/5
646/646 [==============================] - 4s 6ms/step - loss: 0.0833 - acc: 0.9783 - val_loss: 0.1287 - val_acc: 0.9568
Epoch 4/5
646/646 [==============================] - 4s 6ms/step - loss: 0.0929 - acc: 0.9737 - val_loss: 0.1275 - val_acc: 0.9568
Epoch 5/5
646/646 [==============================] - 4s 6ms/step - loss: 0.0634 - acc: 0.9876 - val_loss: 0.1224 - val_acc: 0.9568
